# Numerical integration (quadrature)

Python provides various "black box" routines to integrate a function, e.g., `quad` from `scipy.integrate`:

In [ ]:
import numpy as np

from scipy.integrate import quad

f = lambda x: x**2 * np.cos(x)

print(quad(f, 0.0, 1.0))

These are very convenient to use and in many cases it is not necessary to know how they work internally.

For solving integral equations, however, we need to work directly with a **quadrature rule**:

\begin{equation}
 I[f] = \int_a^b f(x) \, \mathrm{d}x \approx \sum_i w_i f(x_i)
\end{equation}

How could the approximation is depends on both the chosen points ($x_i$) and weights ($w_i$), as well as on properties of the function $f$. In special cases, a quadrature rule can be exact for a finite number of terms. 
In general, the approximation becomes better the more points are included in the sum.

Our library provides a wrapper class for the commonly used **Gauss-Legendre** quadrature rule:

In [ ]:
from lib.mesh import *

mesh = GaulegMesh(16, 0.0, 1.0)

Points are not evenly spaced for this particular mesh, they are concentrated towards the end points:

In [ ]:
print(mesh.ps())

import matplotlib.pyplot as plt

plt.plot(mesh.ps(), 0.0 * mesh.ps(), marker="o")
plt.show()

But points in the middle have larger weight:

In [ ]:
print(mesh.ps())

plt.plot(mesh.ps(), mesh.ws(), marker="o")
plt.show()

We can use the quadrature mesh to integrate again our function `f`:

In [ ]:
acc = 0
for i in range(0, mesh.n):
  acc += mesh.ws()[i] * f(mesh.ps()[i])
  
print(acc)

Let us consider now another integral:

\begin{equation}
 I[g] = \int_0^{100} \frac{\sin(x)}{1+x^2} \,\mathrm{d}x \approx 0.6467
\end{equation}

If we integrate this with a 16-point Gauss-Legendre mesh, the result is still off by about 20%:

In [ ]:
g = lambda x: np.sin(x) / (1.0 + x**2)

mesh1 = GaulegMesh(16, 0.0, 100.0)

acc = 0
for i in range(0, mesh.n):
  acc += mesh1.ws()[i] * g(mesh1.ps()[i])
  
print(acc)

The most obvious way to improve this is to increase the number of mesh points, but we can in fact improve the accuracy another way, *using the same number of points*.

The key is to recognize that due to the denominator, most of the contribution to the integral comes from small $x$.  To account for this, we want to modify our mesh in such a way that there are more points in the small-$x$ region.  One way to achieve this is of course to simply split up the integration domain and discretize each interval separately; this can be useful in cases where physics provides guidance where to put the splitting point.

Lacking that, we can alternatively use a **variable transformation** to change the density of mesh points: let $x=\alpha(y)$ in

\begin{equation}
 I[f] = \int_a^b f(x) \, \mathrm{d}x \,,
\end{equation}

then we can create a standard Gauss-Legendre quadrature on the interval $[\alpha^{-1}(a),\alpha^{-1}(b)]$, yielding $(y_i,w_i)$, and calculate the final points and weights as follows:

\begin{align}
 x_i &= \alpha(y_i) \,, \\
 w_i &\to \alpha'(y_i)\, w_i \,.
\end{align}

A particular choice is $\alpha(y) = \exp(y)-1$, which produces a logarithmic distribution of points.  Our library already provides a mesh class that does exactly this:

In [ ]:
mesh2 = ExpGaulegMesh(16, 0.0, 100.0)

print(mesh2.ps())

plt.plot(mesh2.ps(), mesh2.ws(), marker="o")
plt.show()

And indeed we find that this mesh now brings us within less than 5% of the converged result:

In [ ]:
acc = 0
for i in range(0, mesh.n):
  acc += mesh2.ws()[i] * g(mesh2.ps()[i])
  
print(acc)

Importantly, the same trick can be used to improve convergence when the integrand *a priori* unknown, i.e., when we are solving an integral equation.